## Configs

In [1]:
offset = 0
limit = 3000
period = 'max' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

## Download all NASDAQ traded symbols

In [8]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 11356


## Download Historic data

In [4]:
! pip install yfinance
! mkdir hist

                                              0.0/63.2 kB ? eta -:--:--
     ---------------------------------------- 63.2/63.2 kB 3.5 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
                                              0.0/3.8 MB ? eta -:--:--
     ---                                      0.3/3.8 MB 8.9 MB/s eta 0:00:01
     ------------                             1.2/3.8 MB 12.5 MB/s eta 0:00:01
     -----------------------------            2.8/3.8 MB 19.9 MB/s eta 0:00:01
     ---------------------------------------  3.8/3.8 MB 21.8 MB/s eta 0:00:01
     ---------------------------------------- 3.8/3.8 MB 19.9 MB/s eta 0:00:00
                                              0.0/143.0 kB ? eta -:--:--
     -------------------------------------- 143.0/143.0 kB 8.3 MB/s eta 0:00:00
                                              0.0/112.2 kB ? eta -:--:--
     ---------------------------------------- 112.2/112.2 kB ? eta 0:00:00
  Using cached charset_nor

The syntax of the command is incorrect.


In [9]:
import yfinance as yf
import os, contextlib

In [11]:
%%time

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_csv('hist/{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))


1 Failed download:
['AAC=']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AAC+']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AACIW']: Exception('%ticker%: 1d data not available for startTime=-2208994789 and endTime=1687540676. Only 100 years worth of day granularity data are allowed to be fetched per request.')

1 Failed download:
['AACT=']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AACT+']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AAIC-B']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AAIC-C']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AAM-A']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['AAM-B']: Exception('%ticker%: No timezone found, symbol may be delisted')

1 Failed download:
['

KeyboardInterrupt: 

In [6]:
valid_data = data_clean[is_valid]
valid_data.to_csv('symbols_valid_meta.csv', index=False)

## Separating ETFs and Stocks

In [7]:
!mkdir stocks
!mkdir etfs

In [8]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [9]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join('hist', filename), join(dest, filename))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")

In [10]:
! rmdir hist